In [18]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader

In [19]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [20]:
full_dataset = pd.read_csv('train.csv')
# dataset = full_dataset[full_dataset['sc_w'] != 0].copy()

In [21]:
full_dataset.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [22]:
torch.manual_seed(1)
indices = torch.randperm(len(full_dataset)).tolist()
train_size = int(0.8*len(full_dataset))
dataset_train = full_dataset.iloc[indices[:train_size]]
dataset_test = full_dataset.iloc[indices[train_size:]]

In [23]:
class CreatingDatabase(Dataset):
    def __init__(self,group):
        self.data = group.to_numpy()
        
    def __getitem__(self,idx):

        sample = self.data[idx][0:20]
        label = self.data[idx][-1:]

        sample = torch.from_numpy(sample.astype(np.float32))
        label = torch.LongTensor(label)

        return sample,label

    def __len__(self):
        return len(self.data)


In [24]:
train_set = CreatingDatabase(dataset_train)
test_set = CreatingDatabase(dataset_test)

In [25]:
batch_size = 20
num_workers = 4

train_loader = DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = DataLoader(test_set,batch_size = batch_size,shuffle = True)

# Creating NN Model

Linear ->
ReLU ->
Out ->
Softmax

In [26]:
class PricePredictor(nn.Module):
    def __init__(self,input_size,hidden_size,out_size):
        super(PricePredictor,self).__init__()
        self.hidden = nn.Sequential(nn.Linear(input_size,hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(hidden_size,hidden_size),
                                    nn.ReLU())
        self.out = nn.Linear(hidden_size,out_size)
        self.softmax = nn.Softmax()

    def forward(self,X):
        feature = self.hidden(X)
        output = self.softmax(self.out(feature))
        
        return output

In [27]:
input_size = 20
hidden_size = 128
out_size = 4

In [28]:
net = PricePredictor(input_size,hidden_size,out_size).to(device)

In [29]:
criterion = nn.CrossEntropyLoss().to(device)

In [30]:
optimizer = optim.Adam(net.parameters(),lr=1e-3,weight_decay = 1e-4)

In [31]:
def train(train_loader,net,epoch):

    net.train()

    epoch_loss = []
    for batch in train_loader:
        data, classification = batch

        data = data.to(device)
        classification = classification.to(device)

        pred = net(data)
        loss = criterion(pred,classification)

        epoch_loss.append(loss.cpu().data)

        loss.backward()
        optimizer.step()

    epoch_loss = np.asarray(epoch_loss)
    print(epoch,epoch_loss)

In [32]:
def test(test_loader,net,epoch):

    net.eval()
    
    with torch.no_grad():
            
        epoch_loss = []
        for batch in test_loader:
            data, classification = batch

            data = data.to(device)
            classification = classification.to(device)

            pred = net(data)
            loss = criterion(pred,classification)

            epoch_loss.append(loss.cpu().data)

        epoch_loss = np.asarray(epoch_loss)
        print(epoch,epoch_loss)

In [33]:
for epoch in range(10):
    train(train_loader,net,epoch)
    test(test_loader,net,epoch)
    print('-----------------------------------------------------------')

RuntimeError: 0D or 1D target tensor expected, multi-target not supported